# Wave Equation

## Bryant McArthur

## Math 437

## November 8, 2022

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation as ani
from scipy.linalg import toeplitz
import scipy.optimize

In [2]:
%matplotlib notebook

# Problem 1
Numerically approximate the solution to the following BVP

$$
u_{tt} = u_{xx} \\
u(0,t) = u(1,t) = 0, \\
u(x,0) = \sin(2\pi x), \\
u_t(x,0) = 0. 
$$

Numerically approximate the solution $u(x,t)$ for $t\in[0,5]$. Use $J=50$ subintervals in the $x$ dimension and $M=50$ subintervals in the $t$ dimension. Animate the results. Compare you results with the analytic solution $u(x,t) = \sin(2\pi x)\cos(2\pi t).$ This function is know as a standing wave.

In [6]:
def prob1(t0,T,M,a,b,J,f,fpp=None,g=lambda x:0,s=1):
    
    # set everything up
    t = np.linspace(t0,T,M)
    x = np.linspace(a,b,J)
    u = np.zeros((J,M))
    
    dt = t[1] - t[0]
    dx = x[1] - x[0]
    
    
    # Initial time step
    u[1:-1,0] = f(x[1:-1])
    
    # First time step
    if fpp==None:
        fpp = np.array([u[j-1,0]-2*u[j,0]+u[j+1,0] for j in range(1,J-1)])/dx**2
        u[1:-1,1] = u[1:-1,0] + g(x[1:-1])*dt + s**2*fpp*(dt**2)/2
    else:
        u[1:-1,1] = u[1:-1,0] + g(x[1:-1])*dt + s**2*fpp(x[1:-1])*(dt**2)/2
    
    # Remaining time step
    lamb = s*dt/dx
    A = np.diag(lamb**2*np.ones(J-1), k=-1) + np.diag(2*(1-lamb**2)*np.ones(J), k=0) + np.diag(lamb**2*np.ones(J-1), k=1)
    
    for m in range(2,M):
        u[:,m] = A@u[:,m-1] - u[:,m-2]
        u[0,m] = 0
        u[-1,m] = 0
        
    
    return u,x
    

In [4]:
# Auxillary conditions
J=50
M=50
t = np.linspace(0,5,M)
x = np.linspace(0,1,J)
u = np.zeros((J,M))

f = lambda x: np.sin(2*np.pi*x)
fpp = lambda x: -4*np.pi**2*np.sin(2*np.pi*x)

In [5]:
# Get solution
u,x = prob1(0,5,M,0,1,J,f,fpp)

# plot and animate it
plt.ioff()
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim((0,1))
ax.set_ylim((-1,1))
ax.set_title("Numerical approximation")
#ax.set_ylim((-10,10))
sol, = plt.plot([],[])

def update(i):
    sol.set_data(x,u[:,i])
    return sol

anim = ani.FuncAnimation(fig, update, frames=range(50), interval=20)
# plt.show()
anim.save('problem1a.mp4')

 <video src='problem1a.mp4' controls>

In [6]:
# Auxillary conditions
t = np.linspace(0,5,M)
x = np.linspace(0,1,J)
u = np.zeros((J,M))

# Real solution
f = lambda x,t: np.sin(2*np.pi*x)*np.cos(2*np.pi*t)
for j in range(J):
    for m in range(M):
        u[j,m] = f(x[j],t[m])

# Plot and animate
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim((0,1))
ax.set_ylim((-1,1))
ax.set_title("Actual Solution")
#ax.set_ylim((-10,10))
sol, = plt.plot([],[])

def update(i):
    sol.set_data(x,u[:,i])
    return sol

anim = ani.FuncAnimation(fig, update, frames=range(50), interval=20)
# plt.show()
anim.save('problem1b.mp4')

 <video src='problem1b.mp4' controls>

# Problem 2

Consider the boundary value problem 

$$
u_{tt} = u_{xx}, \\
u(0,t) = u(1,t) = 0, \\
u(x,0) = 0.2e^{-,^2(x-1/2)^2} \\
y_t(x,0) = 0.4m^2(x-\frac{1}{2})e^{-m^2(x-1/2)^2}.
$$

The solution of this problem is ta Gaussian pulse. It travels to the right at a constant speed. This solution models, for example, a wave pulse in a stretched string. Note that the fixed boundart conditions reflect the pulse back when it meets the boundary.

Numerically approximate the solution $u(x,t)$ for $t\in[0,1]$. Set $m=20$. Use 200 subintervals in space and 220 in time, and animate your results. Then use 200 subintervals in space and 180 in time, and animate your results. Note that the stability condition is not satisfied for the second mesh.

In [8]:
#First one
m = 20
f = lambda x: .2*np.exp(-m**2*(x-.5)**2)
g = lambda x: .4*m**2*(x-.5)*np.exp(-m**2*(x-.5)**2)
M = 220
J = 200

# Get solultion
u,x = prob1(0,1,M,0,1,J,f,g=g)

# Plot and animate it
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim((0,1))
ax.set_ylim((-1,1))
ax.set_title("Numerical approximation M=220")
#ax.set_ylim((-10,10))
sol, = plt.plot([],[])

def update(i):
    sol.set_data(x,u[:,i])
    return sol

plt.ioff()
anim = ani.FuncAnimation(fig, update, frames=range(220), interval=10)
# plt.show()

anim.save('problem2a.mp4')

 <video src='problem2a.mp4' controls>

In [8]:
#Second one with M=180
M = 180

# Get soultion
u,x = prob1(0,1,M,0,1,J,f,g=g)

# Plot and animate it
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim((0,1))
ax.set_ylim((-1,1))
ax.set_title("Numerical approximation M=180")
#ax.set_ylim((-10,10))
sol, = plt.plot([],[])

def update(i):
    sol.set_data(x,u[:,i])
    return sol

plt.ioff()
anim = ani.FuncAnimation(fig, update, frames=range(180), interval=10)
# plt.show()
anim.save('problem2b.mp4')

 <video src='problem2b.mp4' controls>

# Problem 3

Consider the initial boundary value problem
$$
	u_{tt} = u_{xx}, \\
	u(0,t) = u(1,t) = 0, \\
	u(x,0) = 0.2e^{-m^2(x-1/2)^2}\\
	u_t(x,0) = 0.
$$
The initial condition separates into two smaller, slower-moving pulses, one travelling to the right and the other to the left.
This solution models, for example, a plucked guitar string

Numerically approximate the solution $u(x,t)$ for $t \in \left[0,2\right]$.
Set $m=20$.
Use 200 subintervals in space and 440 in time, and animate your results.
It is rather easy to see that the solution to this problem is the sum of two travelling waves, one travelling to the left and the other to the right, as described earlier.

In [9]:
# Auxillary conditions
m = 20
f = lambda x: .2*np.exp(-m**2*(x-.5)**2)
M = 440
J = 200

# Get solution
u,x = prob1(0,2,M,0,1,J,f)

# Plot and animate it
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim((0,1))
ax.set_ylim((-1,1))
ax.set_title("Numerical approximation M=440")
#ax.set_ylim((-10,10))
sol, = plt.plot([],[])

def update(i):
    sol.set_data(x,u[:,i])
    return sol

plt.ioff()
anim = ani.FuncAnimation(fig, update, frames=range(440), interval=10)
# plt.show()
anim.save('problem3.mp4')

 <video src='problem3.mp4' controls>

# Problem 4

Consider the initial boundary value problem

$$
	u_{tt} = u_{xx}, \\
	u(0,t) = u(1,t) = 0, \\
	u(x,0) = \begin{cases} 1/3 & \text{if } 5/11 < x < 6/11,\\
	0 & \text{otherwise}
	\end{cases}\\
	u_t(x,0) = 0.
$$

Numerically approximate the solution $u(x,t)$ for $t \in \left[0, 2\right]$.
Use 200 subintervals in space and 440 in time, and animate your results.
Even though the method is second order and stable for this discretization, since the initial condition is discontinuous there are large dispersive errors.

In [10]:
#First one
m = 20
t = np.linspace(0,2,440)
x = np.linspace(0,1,200)

dt = t[1] - t[0]
dx = x[1] - x[0]
#define f from the pde
f = lambda x: np.piecewise(x, [((x > 5/11) & (x < 6/11)), ((x <= 5/11) | (x >= 6/11))], [1/3, 0])
lam = 1 * dt / dx
# g = lambda x: .4*m**2*(x-.5)*np.exp(-m**2*((x-1)/2)**2)
M = 440
J = 200

u,x = prob1(0,2,M,0,1,J,f)

plt.ioff()
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim((0,1))
ax.set_ylim((-1.2,1.2))
ax.set_title("Numerical approximation M=440")
#ax.set_ylim((-10,10))
sol, = plt.plot([],[])

def update(i):
    sol.set_data(x,u[:,i])
    return sol

anim = ani.FuncAnimation(fig, update, frames=range(440), interval=10)
# plt.show()
anim.save('problem4.mp4')

 <video src='problem4.mp4' controls>

# Problem 5


Numerically solve the initial value problem

$$
	u_t -su_x + uu_x = u_{xx}, \quad x \in (-\infty,\infty),\\
	u(x,0) = v(x),
$$

for $t \in [0,1]$.
Let the perturbation $v(x)$ be given by

$$
v(x) = 3.5(\sin{(3x)} + 1)\frac{1}{\sqrt{2\pi}} \exp{(-x^2/2)}
$$

And let the initial condition be $u(x, 0) = \hat{u}(x) + v(x)$
Approximate the $x$ domain,$(-\infty, \infty)$, numerically by the finite interval $[-20,20]$, and fix $u(-20) = u_-$, $u(20) = u_+$. Let $u_- = 5$, $u_+ = 1$.
Use 150 intervals in space and 350 steps in time.
Animate your results.
You should see the solution converge to a translate of the travelling wave $\hat{u}$.

Hint: This difference scheme is no longer a linear equation.
We have a nonlinear equation in $U^{n+1}$.
We can still solve this function using Newton's method or some other similar solver.
In this case, use `scipy.optimize.fsolve`.

In [5]:
def prob5(M,J):
    
    # Auxillary conditions
    dx = 40 / J
    dt = 1 / M
    x = np.linspace(-20, 20, J+1)
    k1 = dt / (4 * dx)
    k2 = dt /(2 * dx**2)
    um = 5
    up = 1
    s = (um + up) / 2
    a = (um - up) / 2

    # Make u
    u = np.zeros((M+1, J+1))
    u[0,0] = um
    u[0,-1] = up

    # define v and uhat
    v = lambda x: 7/2 * (np.sin(3*x) + 1) * (1/np.sqrt(2*np.pi)) * np.exp(-.5*x**2)
    uhat = lambda x: s - a*np.tanh(a*x/2)

    # First row
    u[0][1:J+1] = [uhat(xj) + v(xj) for xj in x[1:J+1]]

    # get f0
    f0 = u[0]

    # Use crank-Nicolson method
    def crank(u1,u0,k1,k2,s):
        ncols = u1.shape[-1]
        root_eq = [u1[i] - u0[i] - k1*((s-u1[i])*(u1[i+1]-u1[i-1]) + (s-u0[i])*(u0[i+1]-u0[i-1])) - k2*((u1[i+1]-2*u1[i] + u1[i-1]) + (u0[i+1]-2*u0[i]+u0[i-1])) for i in range(1, ncols-1)]
        root_eq = [u1[0] - u0[0]] + root_eq + [u1[-1] - u0[-1]]
        return root_eq

    # find roots of crank nicolson equations
    for i in range(1, M+1):
        u[i] = scipy.optimize.fsolve(crank, f0, args=(u[i-1], k1, k2, s))
        
    return u,x



In [7]:
# Get solution
M = 350
J = 150
u,x = prob5(M,J)

# Plot and animate
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim((-20,20))
ax.set_ylim((0,6))
ax.set_title("Crank-Nicolson Approximation")
sol, = plt.plot([],[])
def update(i):
    sol.set_data(x,u[i])
    return sol
    
plt.ioff()
anim = ani.FuncAnimation(fig, update, frames=range(M), interval=10)
anim.save("problem5.mp4")

<video src='problem5.mp4' controls>